In [3]:
from sklearn import tree
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
import numpy as np
from sklearn import metrics

In [10]:
data = pd.read_csv('train.csv', index_col='PassengerId')
test = pd.read_csv('test.csv', index_col='PassengerId')
result = pd.concat([data['Age'],test['Age']])
age_Mean = np.mean(result)

In [12]:
data = pd.read_csv('train.csv', index_col='PassengerId')
#dropping columns that have confusing features
data.drop(['Name', 'Ticket'], axis = 1, inplace = True)
# chaning the Cabin feature to a binary feature of having or not having a cabin
data['hasCabin'] = data['Cabin'].apply(lambda x:0 if type(x) == float else 1)
data.drop(['Cabin'], axis = 1, inplace=True)
# binarizing the sex feature
data['Sex'] = data['Sex'].map({'female':0, 'male':1}).astype(int)
#binarizing the Pclass feature to 1 if 1st class and 0 if 2nd or 3rd class
data['Pclass'] = data['Pclass'].map({1:1, 2:0 , 3:0}).astype(int)
#binarzing the Age features into adults and senior
data['Age'] = data['Age'].apply(lambda x: 0 if x<np.mean(data['Age']) else 1)
#binarizing the Embarked feature to 1 if S 0 otherwise
data['Embarked'] = data['Embarked'].fillna('S')
data['EmbarkedS'] = data['Embarked'].map({'S':1, 'C':0, 'Q':0}).astype(int)
data.drop(['Embarked'], axis = 1, inplace=True)
#binaring the Fare feature to 0 if less than mean and 1 if greater than
data['Fare'] = data['Fare'].apply(lambda x: 0 if x< 31.00 else 1)
# binarizing the Parch and SibSp feature to one feature if isAlone or not
data['FamilySize'] = data['SibSp'] + data['Parch']+1
data['isAlone'] = data['FamilySize'].apply(lambda x: 1 if x==1 else 0)
data.drop(['FamilySize', 'Parch', 'SibSp'], inplace = True, axis = 1)

In [13]:
train_x, test_x, train_y, test_y =train_test_split(data.drop('Survived', axis = 1),data['Survived'], test_size = 0.33, random_state= 42)

In [56]:
clf = BernoulliNB()
clf.fit(train_x, train_y)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [64]:
import math
print([math.exp(x)for x in clf.feature_log_prob_[0]])
print([math.exp(x)for x in clf.feature_log_prob_[1]])

[0.148936170212766, 0.8457446808510638, 0.5851063829787236, 0.18882978723404253, 0.12765957446808512, 0.7898936170212767, 0.6675531914893613]
[0.36160714285714307, 0.3303571428571431, 0.5357142857142858, 0.36607142857142877, 0.3928571428571431, 0.6383928571428571, 0.4955357142857143]


In [15]:
predict_train_x = clf.predict(train_x)
predict_test_x = clf.predict(test_x)
accurary_train = metrics.accuracy_score(predict_train_x, train_y)
accuracy_test = metrics.accuracy_score(predict_test_x, test_y)

In [16]:
print("train accurary is ", accurary_train)
print("test accurary is ", accuracy_test)

train accurary is  0.743288590604
test accurary is  0.769491525424


In [45]:
predicted_prob = clf.predict_proba(test_x)
yesClass = []
noClass = []
for i in range(0,len(predicted_prob)):
    if(predicted_prob[i][0]>predicted_prob[i][1]):
        yesClass.append(i)
    else:
        noClass.append(i)
print(yesClass)
print(noClass)

[0, 1, 2, 7, 11, 13, 14, 18, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 39, 40, 41, 42, 43, 45, 46, 47, 48, 52, 54, 56, 58, 60, 63, 64, 66, 67, 68, 74, 75, 78, 80, 83, 85, 88, 89, 90, 93, 94, 95, 97, 99, 100, 101, 103, 104, 105, 108, 111, 113, 115, 117, 120, 123, 124, 125, 129, 131, 132, 133, 137, 138, 140, 142, 143, 146, 147, 149, 150, 151, 152, 154, 155, 156, 160, 161, 162, 164, 166, 168, 169, 172, 174, 175, 177, 179, 180, 181, 182, 185, 186, 187, 189, 190, 191, 192, 193, 197, 200, 201, 203, 204, 206, 208, 209, 211, 213, 215, 219, 220, 221, 222, 223, 224, 226, 229, 230, 232, 234, 235, 237, 239, 240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 258, 262, 263, 264, 265, 266, 267, 269, 271, 272, 275, 276, 277, 279, 280, 281, 283, 284, 285, 287, 288, 289, 290, 291, 294]
[3, 4, 5, 6, 8, 9, 10, 12, 15, 16, 17, 19, 22, 36, 37, 38, 44, 49, 50, 51, 53, 55, 57, 59, 61, 62, 65, 69, 70, 71, 72, 73, 76, 77, 79, 81, 82, 84, 86, 87, 91, 92, 96, 98, 102

In [53]:
predicted_log = clf.predict_log_proba(test_x)
total_log_evidence_yes = 0
for i in yesClass:
    total_log_evidence_yes+=predicted_log[i][0]-predicted_log[i][1]
total_log_evidence_no = 0
for i in noClass:
    total_log_evidence_no+=predicted_log[i][0]-predicted_log[i][1]
print(total_log_evidence_yes)
print(total_log_evidence_no)

370.515464138
-200.058556653
